## 🔧 1. Installation

You can install PySpark via pip:
```
pip install pyspark
```
To verify the installation, run the following command in Python:

In [43]:
pip install pyspark

In [44]:
import pyspark
print(pyspark.__version__)

3.5.1


Before intializing PySpark, please make sure your computer installed Java 8 or later, but Java 9+ can sometimes cause issues. To check your Java version, run:
```{sh}
java -version
```
You should see something like:
```
java version "1.8.0_281"
```
If Java is not installed, install it trough the link
-  MacBook: [Install Java on MacBook](https://www.java.com/en/download/) (Check if your computer has Intel or Apple M CPU first)
-  Windows: [Install Java on PC](https://www.java.com/download/ie_manual.jsp)

## 🚀 2. Start a Spark Session

In [45]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("PySpark_Tutorial") \
    .master("local[*]") \
    .getOrCreate()

- `SparkSession.builder`: it starts building a new SparkSession configuration; you may treat it as a fluent API (a builder pattern) for customizing your Spark app.
- `.appName("PySpark_Tutorial")`: it sets the name of your Spark application
- `.master("local[*]")`: `"local"` tells Spark to run locally, not on a cluster; `"*"` tells Spark to use all available CPU cores on your machine.`
- `.getOrCreate()`: Creates a new SparkSession or returns an existing one if it already exists.

## 📂 3. Load and Explore Data
The data required for this example is saved at s3://de300spring2025/dinglin_xia/lab7_data/adult.csv.

Assuming your file is named "adult.csv" and contains a column "filtered" (JSON array of words):
We’ll walk through several important steps:

1. Feature Engineering

2. Bias Analysis on Marital Status

3. Joining with Supplemental Gender Data

4. Exporting Final Data

In [46]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col, when, isnan, isnull, count, avg, trim
import os

In [47]:
DATA_FOLDER = "data"
# source https://www.statista.com/statistics/242030/marital-status-of-the-us-population-by-sex/
# the first value is male and the second is for female
MARITAL_STATUS_BY_GENDER = [
    ["Never-married", 47.35, 41.81],
    ["Married-AF-spouse", 67.54, 68.33],
    ["Widowed", 3.58, 11.61],
    ["Divorced", 10.82, 15.09]
]
MARITAL_STATUS_BY_GENDER_COLUMNS = ["marital_status_statistics", "male", "female"]

### 📁 spark.read
- `schema(schema)`: Tells Spark to use your predefined schema.

- `option("header", "false")`: There’s no header row in the CSV file.

- `option("inferSchema", "false")`: You don’t want Spark to guess column types.

- `csv(...)`: Loads the CSV files from the given folder (wildcard *.csv means all CSVs).

### 🧵 Repartition
Repartitions the DataFrame into 8 partitions, which improves parallel processing.

In [70]:
def read_data(spark: SparkSession) -> DataFrame:
    """
    read data based on the given schema; this is much faster than spark determining the schema
    """

    # Define the schema for the dataset
    # It tells Spark: 1. what column name to expect 2. what data type it should have 3. and whether it can contain null
    # It's faster than letting Spark infer types.
    schema = StructType([
        StructField("age", IntegerType(), True),
        StructField("workclass", StringType(), True),
        StructField("fnlwgt", FloatType(), True),
        StructField("education", StringType(), True),
        StructField("education_num", FloatType(), True),
        StructField("marital_status", StringType(), True),
        StructField("occupation", StringType(), True),
        StructField("relationship", StringType(), True),
        StructField("race", StringType(), True),
        StructField("sex", StringType(), True),
        StructField("capital_gain", FloatType(), True),
        StructField("capital_loss", FloatType(), True),
        StructField("hours_per_week", FloatType(), True),
        StructField("native_country", StringType(), True),
        StructField("income", StringType(), True)
    ])

    # Read the dataset
    data = spark.read \
        .schema(schema) \
        .option("header", "false") \
        .option("inferSchema", "false") \
        .csv(os.path.join("*.csv"))

    data = data.repartition(8)

    # This loop finds all columns with FloatType and casts them to IntegerType
    float_columns = [f.name for f in data.schema.fields if isinstance(f.dataType, FloatType)]
    for v in float_columns:
        data = data.withColumn(v, data[v].cast(IntegerType()))

    # Get the names of all StringType columns
    string_columns = [f.name for f in data.schema.fields if isinstance(f.dataType, StringType)]

    # Remove leading and trailing spaces in all string columns
    for column in string_columns:
        data = data.withColumn(column, trim(data[column])) # Applies the trim() function to remove leading/trailing spaces, which is a common issue in raw CSVs.

    # Show the first 5 rows of the dataset
    data.show(5)

    return data

🔧 Breakdown of the query to count missing value
- `data.columns`: List of all column names.

- `isnan(c)`: Checks if a value is NaN (only for numeric types).

- `isnull(c)`: Checks if a value is null (works for any type).

- `when(condition, value)`: Creates a conditional column.

- `count(...)`: Counts how many times that condition is met.

- `.alias(c)`: Names the result column after the original column.

In [71]:
def missing_values(data: DataFrame) -> DataFrame:
    """
    count the number of samples with missing values for each row
    remove such samples
    """
    # Count Missing Values per Column
    missing_values = data.select([count(when(isnan(c) | isnull(c), c)).alias(c) for c in data.columns])

    # Show the missing values count per column
    missing_values.show()

    # Get the number of samples in the DataFrame
    num_samples = data.count()

    # Print the number of samples
    print("Number of samples:", num_samples)

    data = data.dropna()

    return data

### 📌 Feature Engineering Function
📖 Explanation:
This function programmatically finds all integer features and creates new features by multiplying each pair. It helps capture interaction effects for models or exploratory analysis.

- `withColumn()` creates a new column in the DataFrame.

In [72]:
def feature_engineering(data: DataFrame) -> DataFrame:
    """
    Calculate the product of each pair of integer features
    """
    # Identify all integer-type columns in the dataset
    integer_columns = [f.name for f in data.schema.fields if isinstance(f.dataType, IntegerType)]

    # For each pair of integer columns, compute a new column that is their product
    for i, col1 in enumerate(integer_columns):
        for col2 in integer_columns[i:]:  # Avoid duplicate pairs
            product_col_name = f"{col1}_x_{col2}"
            data = data.withColumn(product_col_name, col(col1) * col(col2))

    # Preview first 5 rows to check new columns
    data.show(5)

    return data

### 📌 Analyze Bias by Marital Status
📖 Explanation:
This function investigates if capital_gain varies significantly with marital_status. It also filters and inspects a specific subgroup — divorced individuals — to facilitate further analysis or visualization.
Compute Average Capital Gain by Marital Status
- `groupBy("marital_status")`: Groups the dataset by marital status (e.g., "Married", "Single", "Divorced").

- `agg(...)`: Performs an aggregation, in this case avg("capital_gain").

- `alias(...)`: Renames the resulting column to "average_capital_gain".



In [73]:
def bias_marital_status(data: DataFrame):
    """
    Analyze if there's a bias in capital gain by marital status
    """
    # Group by marital status and compute average capital gain
    average_capital_gain = data.groupBy("marital_status").agg(avg("capital_gain").alias("average_capital_gain"))
    average_capital_gain.show()

    # Filter only rows with marital_status == "Divorced"
    divorced_data = data.filter(data.marital_status == "Divorced")
    divorced_data.show(5)

### 📌 Join with External Gender Statistics
📖 Explanation:
This function enriches the dataset by joining it with an external dataset containing U.S. gender distribution by marital status. The `outer` join ensures we keep unmatched records for completeness.

In [74]:
def join_with_US_gender(spark: SparkSession, data: DataFrame):
    """
    Join with external data about marital status statistics by gender
    """
    # Example data (assumed predefined in MARITAL_STATUS_BY_GENDER & *_COLUMNS)
    us_df = spark.createDataFrame(MARITAL_STATUS_BY_GENDER, MARITAL_STATUS_BY_GENDER_COLUMNS)

    # Outer join on marital_status
    return data.join(us_df, data.marital_status == us_df.marital_status_statistics, 'outer')

### ✅ Main Pipeline
📖 Explanation:
This function ties all preprocessing steps together and writes the result to a CSV file. You can run it with main() in a PySpark environment.

In [75]:
def main():
    spark = SparkSession.builder.appName("Read Adult Dataset").getOrCreate()

    data = read_data(spark)               # Load dataset
    data = missing_values(data)           # Handle missing values (assumed implemented)
    data = feature_engineering(data)      # Add interaction features
    bias_marital_status(data)             # Analyze capital gain by marital status
    data = join_with_US_gender(spark, data)  # Enrich with gender statistics

    data.show(5)                          # Preview final data
    data.write.format('csv').option('header', 'true').mode('overwrite').save('saved.csv')  # Save

In [76]:
main()

+---+----------------+------+------------+-------------+------------------+--------------+-------------+-----+------+------------+------------+--------------+--------------+------+
|age|       workclass|fnlwgt|   education|education_num|    marital_status|    occupation| relationship| race|   sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+----------------+------+------------+-------------+------------------+--------------+-------------+-----+------+------------+------------+--------------+--------------+------+
| 72|Self-emp-not-inc| 52138|   Doctorate|           16|Married-civ-spouse|Prof-specialty|      Husband|White|  Male|           0|        2392|            25| United-States|  >50K|
| 45|         Private|140581|Some-college|           10|           Widowed| Other-service|    Unmarried|Black|Female|           0|           0|            40| United-States| <=50K|
| 55|       Local-gov| 84564| Prof-school|           15|          Divorced|Prof-specialty|Not-i

## Lab Assignment
### 1.  Spark-sql
Add one more cell in to select rows with 'age' between 30 and 50 (inclusive) and transforms the selected pyspark dataframe into pandas dataframe and print out the summary statistics using 'describe()'. Some useful methods are:
- `filter(...)`: filters Spark DataFrame rows using boolean conditions
- `.toPandas()`: collects the Spark DataFrame into memory, so only use it on small to medium datasets
- `.describe()`: gives summary stats (mean, std, min, max, quartiles)

In [77]:
def describe_age_range(data: DataFrame, lower: int = 30, upper: int = 50):
    """
    Filters rows where age is between `lower` and `upper` (inclusive),
    converts to pandas DataFrame, and prints summary statistics.
    """
    # Filter Spark DataFrame
    filtered_data = data.filter((data.age >= lower) & (data.age <= upper))

    # Convert to pandas DataFrame
    pandas_df = filtered_data.toPandas()

    # Print summary statistics
    print(f"Summary statistics for age between {lower} and {upper}:")

    return pandas_df  # Optional: return if you want to use it later

In [78]:
def main():
    spark = SparkSession.builder.appName("Read Adult Dataset").getOrCreate()

    data = read_data(spark)               # Load dataset
    data = missing_values(data)           # Handle missing values (assumed implemented)
    data = feature_engineering(data)      # Add interaction features
    bias_marital_status(data)             # Analyze capital gain by marital status
    data = join_with_US_gender(spark, data)  # Enrich with gender statistics

    data_df = describe_age_range(data)    # Select and Transform

    data.show(5)                          # Preview final data
    data.write.format('csv').option('header', 'true').mode('overwrite').save('saved.csv')  # Save

In [79]:
main()

+---+---------+------+---------+-------------+------------------+--------------+-------------+-----+------+------------+------------+--------------+--------------+------+
|age|workclass|fnlwgt|education|education_num|    marital_status|    occupation| relationship| race|   sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+---------+------+---------+-------------+------------------+--------------+-------------+-----+------+------------+------------+--------------+--------------+------+
| 34|        ?| 24504|  HS-grad|            9|     Never-married|             ?|Not-in-family|White|  Male|           0|           0|            40| United-States| <=50K|
| 22|  Private|305781|  HS-grad|            9|     Never-married|         Sales|Not-in-family|White|  Male|           0|           0|            45|        Canada| <=50K|
| 50|  Private|175804|Bachelors|           13|Married-civ-spouse|         Sales|      Husband|White|  Male|           0|           0|            

In [58]:
spark.stop()

### 2. Word Count
Save only the words that have count greater or equal to 3.

#### 1. Setup Spark Session

In [59]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("WordCount") \
    .master("local[*]") \
    .getOrCreate()

#### 2. Read Text File
Fill in your text file direction

In [60]:
# Each line becomes a row in the DataFrame
lines = spark.read.text("input.txt")
lines.show()

+--------------------+
|               value|
+--------------------+
|MapReduce is a pr...|
|It is inspired by...|
|The Map function ...|
|while the Reduce ...|
|MapReduce enables...|
|This approach is ...|
+--------------------+



#### 3. Split Lines into Words

In [61]:
from pyspark.sql.functions import split, explode, col

# Split lines into arrays of words, then explode into one word per row
words = lines.select(explode(split(col("value"), " ")).alias("word"))
words.show()

+-----------+
|       word|
+-----------+
|  MapReduce|
|         is|
|          a|
|programming|
|      model|
|        for|
| processing|
|      large|
|       data|
|      sets.|
|         It|
|         is|
|   inspired|
|         by|
|        the|
|        map|
|        and|
|     reduce|
|  functions|
|   commonly|
+-----------+
only showing top 20 rows



#### 4. Count Word Frequencies
Fill in the column name

In [62]:
word_counts = words.groupBy("word").count()
word_counts.show()

+------------+-----+
|        word|count|
+------------+-----+
|         map|    1|
|        used|    2|
|programming.|    1|
|   MapReduce|    2|
|     element|    1|
|  functional|    1|
|          by|    1|
|  aggregates|    1|
|      widely|    1|
|         log|    1|
|         for|    1|
|          in|    2|
| programming|    1|
| collection,|    1|
|       model|    1|
| distributed|    1|
|   machines.|    1|
|     applies|    1|
|          is|    3|
|       sets.|    1|
+------------+-----+
only showing top 20 rows



#### 5. Filter Rare Words
Fill in the condition in `filter()`

In [63]:
from pyspark.sql.functions import length
word_counts_filtered = word_counts.filter(length(col('word')) >= 3)
word_counts_filtered.show()

+------------+-----+
|        word|count|
+------------+-----+
|         map|    1|
|        used|    2|
|programming.|    1|
|   MapReduce|    2|
|     element|    1|
|  functional|    1|
|  aggregates|    1|
|      widely|    1|
|         log|    1|
|         for|    1|
| programming|    1|
| collection,|    1|
|       model|    1|
| distributed|    1|
|   machines.|    1|
|     applies|    1|
|       sets.|    1|
|        each|    1|
|    results.|    1|
|        data|    2|
+------------+-----+
only showing top 20 rows



#### 6. Stop the Spark Session

In [64]:
spark.stop()